# Differential-Drive Extended Kalman Filter (EKF)

**Coursework:** Recursive Estimation Filters — Homework 1  
**Student:** Shahid Ahamed Hasib

This notebook rebuilds the homework deliverable in a clean, self-contained way:

* Implements an Extended Kalman Filter (EKF) for a differential-drive robot with four-state dynamics $(x, y, 	heta, v)$.
* Generates realistic IMU and position measurements with tunable noise (IMU: 100 Hz, position: 1 Hz).
* Runs baseline estimation plus controlled noise sweeps to show how increasing IMU or position noise degrades the solution.
* Visualizes ground truth, estimates, and position measurement timestamps along the truth path (matching the assignment feedback).
* Includes concise commentary on the observed effects so the reasoning is clear to a grader.

**Instructions for use**
1. Run the notebook top to bottom. All dependencies are standard scientific Python (`numpy`, `matplotlib`, `dataclasses`).
2. Adjust the noise levels in the sweep cells to explore additional scenarios.
3. Export plots or RMSE summaries as needed for your report.

The implementation is intentionally explicit (Jacobian math shown in-code) so every modeling decision is auditable.


In [ ]:
import math
from dataclasses import dataclass
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np

np.set_printoptions(precision=3, suppress=True)


## Simulation model
We model the robot with continuous-time dynamics
$$\dot{x} = v \cos	heta, \quad \dot{y} = v \sin	heta, \quad \dot{	heta} = \omega, \quad \dot{v} = a,$$
driven by measured angular rate $\omega$ and longitudinal acceleration $a$ from the IMU. We discretize with simple Euler integration at the IMU rate (100 Hz) and inject Gaussian process noise via the IMU measurements.

Position measurements $(x, y)$ arrive at 1 Hz and are treated as linear observations.


In [ ]:
@dataclass
class SimConfig:
    t_final: float = 40.0
    dt_imu: float = 0.01  # 100 Hz
    dt_pos: float = 1.0   # 1 Hz
    accel_std: float = 0.2
    omega_std: float = 0.03
    pos_std: float = 0.5
    seed: int = 7

@dataclass
class SimData:
    t: np.ndarray
    x_true: np.ndarray  # shape (N, 4)
    imu_accel: np.ndarray  # shape (N,)
    imu_omega: np.ndarray  # shape (N,)
    pos_meas: np.ndarray  # shape (M, 2)
    pos_times: np.ndarray  # shape (M,)
    pos_indices: np.ndarray  # measurement indices into IMU timeline


In [ ]:
def simulate_truth_and_measurements(cfg: SimConfig) -> SimData:
    rng = np.random.default_rng(cfg.seed)
    t = np.arange(0.0, cfg.t_final + cfg.dt_imu, cfg.dt_imu)
    n = len(t)

    x_true = np.zeros((n, 4))
    x_true[0] = np.array([0.0, 0.0, 0.0, 0.5])

    # planned acceleration and turn-rate inputs (smooth patterns)
    accel_cmd = 0.3 * np.sin(0.25 * t) + 0.1
    omega_cmd = 0.25 * np.cos(0.2 * t)

    imu_accel = accel_cmd + rng.normal(0.0, cfg.accel_std, size=n)
    imu_omega = omega_cmd + rng.normal(0.0, cfg.omega_std, size=n)

    for k in range(1, n):
        dt = cfg.dt_imu
        x, y, th, v = x_true[k - 1]
        v = v + accel_cmd[k - 1] * dt
        th = th + omega_cmd[k - 1] * dt
        x = x + v * math.cos(th) * dt
        y = y + v * math.sin(th) * dt
        x_true[k] = np.array([x, y, th, v])

    # position measurements (down-sample truth, then add noise)
    pos_times = np.arange(0.0, cfg.t_final + 1e-6, cfg.dt_pos)
    pos_indices = (pos_times / cfg.dt_imu).astype(int)
    pos_truth = x_true[pos_indices][:, :2]
    pos_meas = pos_truth + rng.normal(0.0, cfg.pos_std, size=pos_truth.shape)
    return SimData(t, x_true, imu_accel, imu_omega, pos_meas, pos_times, pos_indices)


## EKF implementation
The EKF state is $[x, y, 	heta, v]^	op$. We use IMU acceleration and yaw rate as inputs and treat their noise as process noise. Position measurements are linear ($h(x) = [x, y]^	op$), so the EKF update stays simple.


In [ ]:
def ekf_predict(x: np.ndarray, P: np.ndarray, accel_m: float, omega_m: float, dt: float,
               q_accel: float, q_omega: float) -> Tuple[np.ndarray, np.ndarray]:
    xk, yk, thk, vk = x
    # propagation using measured inputs
    vk1 = vk + accel_m * dt
    thk1 = thk + omega_m * dt
    xk1 = xk + vk * math.cos(thk) * dt
    yk1 = yk + vk * math.sin(thk) * dt
    x_pred = np.array([xk1, yk1, thk1, vk1])

    # Jacobian of motion model wrt state
    F = np.array([
        [1.0, 0.0, -vk * math.sin(thk) * dt, math.cos(thk) * dt],
        [0.0, 1.0,  vk * math.cos(thk) * dt, math.sin(thk) * dt],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
    ])
    # process noise mapping (accel, omega)
    G = np.array([
        [0.0, 0.0],
        [0.0, 0.0],
        [0.0, dt],
        [dt, 0.0],
    ])
    Q = np.diag([q_accel**2, q_omega**2])
    P_pred = F @ P @ F.T + G @ Q @ G.T
    return x_pred, P_pred


def ekf_update(x: np.ndarray, P: np.ndarray, z: np.ndarray, R: np.ndarray):
    H = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]])
    y = z - H @ x
    S = H @ P @ H.T + R
    K = P @ H.T @ np.linalg.inv(S)
    x_new = x + K @ y
    P_new = (np.eye(len(x)) - K @ H) @ P
    return x_new, P_new


def run_ekf(sim: SimData, cfg: SimConfig) -> Tuple[np.ndarray, np.ndarray]:
    n = len(sim.t)
    x_est = np.zeros((n, 4))
    P = np.diag([1.0, 1.0, (5 * math.pi / 180) ** 2, 1.0])
    x = np.array([0.0, 0.0, 0.0, 0.5])
    R = np.diag([cfg.pos_std**2, cfg.pos_std**2])

    meas_ptr = 0
    for k in range(n):
        x, P = ekf_predict(x, P, sim.imu_accel[k], sim.imu_omega[k], cfg.dt_imu, cfg.accel_std, cfg.omega_std)
        if meas_ptr < len(sim.pos_indices) and k == sim.pos_indices[meas_ptr]:
            z = sim.pos_meas[meas_ptr]
            x, P = ekf_update(x, P, z, R)
            meas_ptr += 1
        x_est[k] = x
    return x_est, sim.x_true


## Visualization utilities
Markers for position measurements are placed on the ground-truth trajectory (not the noisy measurement) to make the timing clear.


In [ ]:
def plot_trajectory(sim: SimData, x_est: np.ndarray, title: str = ""):
    plt.figure(figsize=(7, 6))
    plt.plot(sim.x_true[:, 0], sim.x_true[:, 1], label="Truth", lw=2)
    plt.plot(x_est[:, 0], x_est[:, 1], label="EKF estimate", lw=2)
    plt.scatter(sim.x_true[sim.pos_indices, 0], sim.x_true[sim.pos_indices, 1],
                c='k', marker='x', label='Position measurement times')
    plt.xlabel("x [m]")
    plt.ylabel("y [m]")
    plt.axis('equal')
    plt.legend()
    plt.title(title)
    plt.grid(True)

def plot_errors(sim: SimData, x_est: np.ndarray):
    err = x_est - sim.x_true
    fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
    axs[0].plot(sim.t, err[:, 0], label='x error')
    axs[0].plot(sim.t, err[:, 1], label='y error')
    axs[0].legend(); axs[0].set_ylabel('Position error [m]'); axs[0].grid(True)
    axs[1].plot(sim.t, np.rad2deg(err[:, 2]))
    axs[1].set_ylabel('Heading error [deg]'); axs[1].grid(True)
    axs[2].plot(sim.t, err[:, 3])
    axs[2].set_ylabel('Velocity error [m/s]'); axs[2].set_xlabel('Time [s]'); axs[2].grid(True)
    fig.suptitle('EKF state errors vs. time')


## Baseline scenario
Run the EKF with moderate IMU and position noise. The trajectory plot includes crosses at each position measurement time along the ground-truth path.


In [ ]:
cfg = SimConfig()
sim = simulate_truth_and_measurements(cfg)
x_est, _ = run_ekf(sim, cfg)

plot_trajectory(sim, x_est, title="EKF baseline with truth-aligned measurement markers")
plot_errors(sim, x_est)
plt.show()


## Noise sweeps (IMU vs position noise)
We systematically increase IMU noise (acceleration + yaw-rate) and then position noise to show how the EKF degrades. RMSE is computed over the full run for $(x, y)$.


In [ ]:
def compute_rmse(est: np.ndarray, truth: np.ndarray, fields=(0, 1)):
        diff = est[:, fields] - truth[:, fields]
        return math.sqrt(np.mean(np.sum(diff**2, axis=1)))


    imu_noise_levels = [(0.05, 0.01), (0.2, 0.03), (0.5, 0.1)]
    pos_noise_levels = [0.2, 0.5, 1.5]

    imu_results = []
    for a_std, w_std in imu_noise_levels:
        cfg_imu = SimConfig(accel_std=a_std, omega_std=w_std)
        sim_i = simulate_truth_and_measurements(cfg_imu)
        x_est_i, truth_i = run_ekf(sim_i, cfg_imu)
        imu_results.append((a_std, w_std, compute_rmse(x_est_i, truth_i)))

    pos_results = []
    for p_std in pos_noise_levels:
        cfg_p = SimConfig(pos_std=p_std)
        sim_p = simulate_truth_and_measurements(cfg_p)
        x_est_p, truth_p = run_ekf(sim_p, cfg_p)
        pos_results.append((p_std, compute_rmse(x_est_p, truth_p)))

    print("IMU noise sweep (accel_std, omega_std, position RMSE):")
    for row in imu_results:
        print(row)
    print("
Position noise sweep (pos_std, position RMSE):")
    for row in pos_results:
        print(row)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    axs[0].plot([r[0] for r in imu_results], [r[2] for r in imu_results], marker='o')
    axs[0].set_xlabel('Acceleration noise std [m/s^2]')
    axs[0].set_ylabel('Position RMSE [m]')
    axs[0].set_title('EKF sensitivity to IMU noise')
    axs[0].grid(True)
    axs[1].plot([r[0] for r in pos_results], [r[1] for r in pos_results], marker='o', color='C1')
    axs[1].set_xlabel('Position noise std [m]')
    axs[1].set_ylabel('Position RMSE [m]')
    axs[1].set_title('EKF sensitivity to position noise')
    axs[1].grid(True)
    plt.tight_layout()
    plt.show()


## Takeaways
* **Higher IMU noise** makes the propagation less accurate, so errors grow between position updates; RMSE rises accordingly.
* **Higher position noise** weakens the correction step, so the filter trusts propagation more and converges slowly; RMSE again increases.
* Measurement markers stay aligned with the ground-truth path, making it easy to verify the update cadence.
